In [1]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
import datetime
import pickle
import pyarrow as pa
import pyarrow.compute as pc
from csutils import *

In [2]:
# Timestamp conversion copied from buses-data-demo.ipynb
VACT_EPOCH = datetime.datetime(year=2011,month=6,day=16,hour=5,minute=23,second=0)
VACT_TIMESTAMP = 1308194580
assert datetime.datetime.fromtimestamp(VACT_TIMESTAMP) == VACT_EPOCH
MILISECOND = 1
SECOND = 1000 * MILISECOND
HOUR = 3600*SECOND
DAY = 24*HOUR
def getVactDate(value):
    return VACT_EPOCH + datetime.timedelta(milliseconds=int(value))

In [3]:
DATA_369 = pickle.load(open("../data/bus/data-369.pickle","rb"))
table = pa.Table.from_pandas(DATA_369)
DATA_369 = table.to_pandas()

DATA_370 = pickle.load(open("../data/bus/data-369.pickle","rb"))
table = pa.Table.from_pandas(DATA_370)
DATA_370 = table.to_pandas()

DATA_371 = pickle.load(open("../data/bus/data-369.pickle","rb"))
table = pa.Table.from_pandas(DATA_371)
DATA_371 = table.to_pandas()

DATA_372 = pickle.load(open("../data/bus/data-369.pickle","rb"))
table = pa.Table.from_pandas(DATA_372)
DATA_372 = table.to_pandas()

DATA_375 = pickle.load(open("../data/bus/data-369.pickle","rb"))
table = pa.Table.from_pandas(DATA_375)
DATA_375 = table.to_pandas()

In [4]:
def df_processing(df):
    df.insert(0, 'tstamp', df.Timestamp.apply(getVactDate))
    df.set_index('tstamp',inplace=True)
    df.sort_index(inplace = True)
    df.drop(columns=['Timestamp'], inplace = True)

In [5]:
for df in [DATA_369,DATA_370,DATA_371,DATA_372,DATA_375]:
    df_processing(df)

In [6]:
# Representations calculations
def calculate_all_reps(DATA):
    new_columns = []
    for name in DATA.columns:
        new_columns.append(f'Numeric_{name}')
        new_columns.append(f'HistBins_{name}')
        new_columns.append(f'HistVals_{name}')
        new_columns.append(f'HistDens_{name}')
        new_columns.append(f'TimeSeries_{name}')
    
    RESULTS = pd.DataFrame(columns=new_columns, dtype=object)
    RESULTS.convert_dtypes(dtype_backend='pyarrow')
    START = DATA.index.min()
    END = DATA.index.max()
    
    FREQ = pd.Timedelta('4T')
    WINDOW_SIZE = pd.Timedelta('4T')
    
    def bin10(column): return 10
    AVG_SIZE = '1S'
    
    for t,win in extract_time_windows(DATA, WINDOW_SIZE, start=START, end=END, freq=FREQ, timestamp = True):
        n = win.shape[0]
        if n < 20:
            continue
        numeric = get_numeric_from_window(win)
        histogram = get_histograms_from_window(win,density=True,way_of_binarization=bin10)
        tseries = get_time_series_from_window(win,t,WINDOW_SIZE,AVG_SIZE=AVG_SIZE)
        for i, name in enumerate(DATA.columns):
            RESULTS.loc[t,f'Numeric_{name}'] = numeric[i]
            RESULTS.loc[t,f'HistBins_{name}'] = histogram[0][i]
            RESULTS.loc[t,f'HistVals_{name}'] = histogram[1][i]
            RESULTS.loc[t,f'HistDens_{name}'] = histogram[2][i]
            RESULTS.loc[t,f'TimeSeries_{name}'] = tseries[i]
    
    return RESULTS

In [7]:
RDF_369 = calculate_all_reps(DATA_369)

/home/tesla/Projects/RISL/Isolation-Similarity-Forest/notebooks/csutils.py:27: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness = skew(window[sensor])
/home/tesla/Projects/RISL/Isolation-Similarity-Forest/notebooks/csutils.py:28: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurt = kurtosis(window[sensor])


KeyboardInterrupt: 

In [ ]:
RDF_369.dtypes

In [ ]:
RDF_369.to_parquet('REP_369.parquet')